In [3]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
# ====================================================================================================
import io
import requests

# URL of the CSV file
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"

# Fetch the CSV file
response = requests.get(url)
response.raise_for_status()  # Check if the request was successful

# Read the CSV content into a pandas DataFrame
data = pd.read_csv(io.StringIO(response.text))
# ====================================================================================================

# Create a Dash application
app = dash.Dash(__name__)
app.title = "Automobile Sales Statistics Dashboard"

# Define the layout of the app
app.layout = html.Div([
    html.H1("Automobile Sales Statistics Dashboard", 
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 24}),
    
    dcc.Dropdown(id='dropdown-statistics', 
                 options=[
                     {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                     {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
                 ],
                 placeholder='Select a report type',
                 value='Select Statistics',
                 style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}),
    
    dcc.Dropdown(id='select-year', 
                 options=[{'label': i, 'value': i} for i in range(1980, 2024, 1)],
                 placeholder='Select a year',
                 style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}),
    
    html.Div([
        html.Div(id='output-container', className='chart-grid', style={'display': 'flex'}),
    ])
])

# Callback to enable or disable the year dropdown based on the selected report
@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown-statistics', component_property='value')
)
def update_input_container(selected_report):
    if selected_report == 'Yearly Statistics': 
        return False
    else: 
        return True

# Callback to update the output container with plots
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='dropdown-statistics', component_property='value'),
     Input(component_id='select-year', component_property='value')]
)
def update_output_container(selected_report, selected_year):
    if selected_report == 'Recession Period Statistics':
        # Filter the data for recession periods
        recession_data = data[data['Recession'] == 1]
        
        # Plot 1: Line chart for Automobile sales fluctuate over Recession Period (year-wise)
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec, 
                x='Year',
                y='Automobile_Sales',
                title="Automobile Sales Fluctuate over Recession Period (Year-wise)")
        )
        
        # Plot 2: Bar chart for the average number of vehicles sold by vehicle type
        avg_vehicle_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(
            figure=px.bar(avg_vehicle_sales, 
                x='Vehicle_Type',
                y='Automobile_Sales',
                title="Average Number of Vehicles Sold by Vehicle Type")
        )
        
        # Plot 3: Pie chart for total expenditure share by vehicle type during recessions
        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
            figure=px.pie(exp_rec,
                values='Advertising_Expenditure',
                names='Vehicle_Type',
                title="Total Expenditure Share by Vehicle Type during Recessions")
        )
        
        # Plot 4: Bar chart for the effect of unemployment rate on vehicle type and sales
        unemployment_sales = recession_data.groupby(['Vehicle_Type', 'Year'])[['unemployment_rate', 'Automobile_Sales']].mean().reset_index()
        R_chart4 = dcc.Graph(
            figure=px.bar(unemployment_sales,
                x='unemployment_rate',
                y='Automobile_Sales',
                color='Vehicle_Type',
                title="Effect of Unemployment Rate on Vehicle Type and Sales")
        )
        
        return [
            html.Div(className='chart-item', children=[html.Div(children=R_chart1), html.Div(children=R_chart2)]),
            html.Div(className='chart-item', children=[html.Div(children=R_chart3), html.Div(children=R_chart4)])
        ]
    
    elif selected_report == 'Yearly Statistics' and selected_year is not None:
        # Filter the data for the selected year
        yearly_data = data[data['Year'] == selected_year]
        
        # Plot 1: Yearly Automobile sales using line chart for the whole period
        yas = data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(
            figure=px.line(yas, 
                x='Year', 
                y='Automobile_Sales', 
                title="Yearly Automobile Sales")
        )
        
        # Plot 2: Total Monthly Automobile sales using line chart
        monthly_sales = yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        Y_chart2 = dcc.Graph(
            figure=px.line(monthly_sales, 
                x='Month', 
                y='Automobile_Sales', 
                title=f"Total Monthly Automobile Sales in {selected_year}")
        )
        
        # Plot 3: Bar chart for average number of vehicles sold during the given year
        avr_vdata = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph(
            figure=px.bar(avr_vdata, 
                x='Vehicle_Type', 
                y='Automobile_Sales', 
                title=f"Average Vehicles Sold by Vehicle Type in the year {selected_year}")
        )
        
        # Plot 4: Total Advertisement Expenditure for each vehicle using pie chart
        exp_data = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(
            figure=px.pie(exp_data, 
                values='Advertising_Expenditure', 
                names='Vehicle_Type', 
                title=f"Total Advertisement Expenditure for Each Vehicle Type in {selected_year}")
        )
        
        return [
            html.Div(className='chart-item', children=[html.Div(children=Y_chart1), html.Div(children=Y_chart2)], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=Y_chart3), html.Div(children=Y_chart4)], style={'display': 'flex'})
        ]
    
    return []

# Run the application
if __name__ == '__main__':
    app.run_server(debug=True)
